# 遗传算法解决旅行商问题（详细解释+代码分享）

链接：<https://zhuanlan.zhihu.com/p/344588977>


In [ ]:
import numpy as np
import math
import time
import pandas as pd

In [ ]:
# def load_data(data_path):
#     """
#     导入数据，得到城市坐标信息
#     :param data_path: 数据文件地址 str
#     :return: 所有城市的坐标信息 二维 list
#     """
#     cities = []
#     with open(data_path, 'r') as f:
#         lines = f.readlines()
#         for line in lines:
#             x_str, y_str = line.split(',')[1:]
#             x, y = int(x_str), int(y_str)
#             cities.append((x, y))
#     return cities

In [ ]:
def get_two_cities_dist(city1, city2):
    """
    计算两个城市的距离
    :param city1: 第一个城市 长度为2的list
    :param city2: 第二个城市 长度为2的list
    :return: 两城市的距离 double
    """
    x_1, y_1 = city1 # 城市的坐标
    x_2, y_2 = city2
    return math.sqrt(math.pow(x_1 - x_2, 2) + math.pow(y_1 - y_2, 2)) # 计算$\sqrt{(x_1 - x_2)^2+(y_1 - y_2)^2}$

In [ ]:
def get_cities_distance(cities):
    """
    计算两两城市的距离
    :param cities: 所有城市的坐标 二维 list
    :return: 城市距离矩阵 numpy数组
    """
    dist_matrix = np.zeros((len(cities), len(cities))) # 生成初始全部为0的$n \times n$阵列
    n_cities = len(cities)
    for i in range(n_cities - 1):
        for j in range(i + 1, n_cities):
            # 循环选择点位
            dist = get_two_cities_dist(cities[i], cities[j]) # 计算两个城市的距离
            dist_matrix[i, j] = dist # 放入相应点位
            dist_matrix[j, i] = dist # 放入斜对称点位
    return dist_matrix

In [ ]:
def get_route_fitness_value(route, dist_matrix):
    """
    计算某一路线的适应度
    :param route: 路线 长度为城市个数的 ndarray
    :param dist_matrix: 距离矩阵 ndarray
    :return: 路线的适应度 double
    """
    dist_sum = 0 # 默认距离总和为0
    for i in range(len(route) - 1):
        # 循环选择路径中的前$n-1$个点位
        dist_sum += dist_matrix[route[i], route[i + 1]] # 将第$i$个和第$i+1$个城市之间的距离加入距离总和
    # dist_sum += dist_matrix[route[len(route) - 1], route[0]] #加入第$n$个和第$0$个城市的距离
    return 1 / dist_sum # 距离总和的倒数即为该条路径的适应度

In [ ]:
def get_all_routes_fitness_value(routes, dist_matrix):
    """
    计算所有路线的适应度
    :param routes: 所有路线 ndarray
    :param dist_matrix: 距离矩阵 ndarray
    :return: 所有路线的适应度 ndarray
    """
    fitness_values = np.zeros(len(routes))
    for i in range(len(routes)):
        f_value = get_route_fitness_value(routes[i], dist_matrix)
        fitness_values[i] = f_value
    return fitness_values

In [ ]:
def init_route(n_route, n_cities):
    """
    随机初始化路线
    :param n_route: 初始化的路线数量 int
    :param n_cities: 城市的数量 int
    :return: 路线矩阵 二维ndarray
    """
    routes = np.zeros((n_route, n_cities)).astype(int) # 初始化**路线数量 x 城市数量**的矩阵
    for i in range(n_route):
        routes[i] = np.random.choice(range(n_cities), size=n_cities, replace=False) # 随机生成乱序的路径，可以用`np.random.shuffle`
    return routes

In [ ]:
def selection(routes, fitness_values):
    """
    选择操作
    :param routes: 所有路线 ndarray
    :param fitness_values: 所有路线的适应度 ndarray
    :return: 选择后的所有路线 ndarray
    """
    selected_routes = np.zeros(routes.shape).astype(int) # 初始化选择到的路径矩阵
    probability = fitness_values / np.sum(fitness_values) # 将适应度转化为选择概率
    n_routes = routes.shape[0]
    for i in range(n_routes):
        choice = np.random.choice(range(n_routes), p=probability) # 按选择概率选择出路径
        selected_routes[i] = routes[choice] # 放置到路径矩阵中相应的位置
    return selected_routes

In [ ]:
def crossover(routes, n_cities):
    """
    交叉操作
    :param routes: 所有路线 ndarray
    :param n_cities: 城市数量 int
    :return: 交叉后的所有路线 ndarray
    """
    for i in range(0, len(routes), 2):
        r1_new, r2_new = np.zeros(n_cities), np.zeros(n_cities) # 初始化两条空白的路径
        seg_point = np.random.randint(0, n_cities) # 随机生成限值的整数，路径从这个点开始交叉
        cross_len = n_cities - seg_point # 参与交叉的路径长度
        r1, r2 = routes[i], routes[i + 1] # 相邻的两条路径
        r1_cross, r2_cross = r2[seg_point:], r1[seg_point:] # 交叉的路径片段
        r1_non_cross = r1[np.in1d(r1, r1_cross, invert=True)] # 选出未参与交叉的路径片段
        r2_non_cross = r2[np.in1d(r2, r2_cross, invert=True)]
        r1_new[:cross_len], r2_new[:cross_len] = r1_cross, r2_cross # 将交叉得到的路径片段放置在空白路径的前方
        r1_new[cross_len:], r2_new[cross_len:] = r1_non_cross, r2_non_cross # 将未参与交叉的路径片段放置在空白路径的后方
        routes[i], routes[i + 1] = r1_new, r2_new # 将交叉完成的两条新路径放置回原来的位置
    return routes

In [ ]:
def mutation(routes, n_cities):
    """
    变异操作，变异概率为 0.01
    :param routes: 所有路线 ndarray
    :param n_cities: 城市数量 int
    :return: 变异后的所有路线 ndarray
    """
    prob = 0.01 # 变异概率
    p_rand = np.random.rand(len(routes)) # 随机生成每条路径变异概率的ndarray
    for i in range(len(routes)):
        if p_rand[i] < prob:
            # 若每条路径变异的概率小于指定值，则变异
            mut_position = np.random.choice(range(n_cities), size=2, replace=False) # 随机选择两个变异点的位置
            l, r = mut_position[0], mut_position[1]
            routes[i, l], routes[i, r] = routes[i, r], routes[i, l] # 两个变异点交换
    return routes

In [ ]:
# 主程序

start = time.time()
n_routes = 100  # 路线
epoch = 100000  # 迭代次数

In [ ]:
# cities_ = load_data('./cities.csv')  # 导入数据
cities_list = [
(9860, 14152),
(9396, 14616),
(11252, 14848),
(11020, 13456),
(9512, 15776),
(10788, 13804),
(10208, 14384),
(11600, 13456),
(11252, 14036),
(10672, 15080),
(11136, 14152),
(9860, 13108),
(10092, 14964),
(9512, 13340),
(10556, 13688),
(9628, 14036),
(10904, 13108),
(11368, 12644),
(11252, 13340),
(10672, 13340),
(11020, 13108),
(11020, 13340),
(11136, 13572),
(11020, 13688),
(8468, 11136),
(8932, 12064),
(9512, 12412),
(7772, 11020),
(8352, 10672),
(9164, 12876),
(9744, 12528),
(8352, 10324),
(8236, 11020),
(8468, 12876),
(8700, 14036),
(8932, 13688),
(9048, 13804),
(8468, 12296),
(8352, 12644),
(8236, 13572),
(9164, 13340),
(8004, 12760),
(8584, 13108),
(7772, 14732),
(7540, 15080),
(7424, 17516),
(8352, 17052),
(7540, 16820),
(7888, 17168),
(9744, 15196),
(9164, 14964),
]

cities = np.array(cities_list)

In [ ]:
dist_matrix = get_cities_distance(cities)  # 计算城市距离矩阵

In [ ]:
routes = init_route(n_routes, dist_matrix.shape[0])  # 初始化所有路线

In [ ]:
fitness_values = get_all_routes_fitness_value(routes, dist_matrix)  # 计算所有初始路线的适应度

In [ ]:
best_index = fitness_values.argmax() # 所有初始路线中适应度最大的路线所在的索引

In [ ]:
best_route, best_fitness = routes[best_index], fitness_values[best_index]  # 保存最优路线及其适应度

In [ ]:
# 开始迭代
not_improve_time = 0 # 未进化次数
for i in range(epoch):
    routes = selection(routes, fitness_values)  # 选择
    routes = crossover(routes, len(cities))  # 交叉
    routes = mutation(routes, len(cities))  # 变异
    fitness_values = get_all_routes_fitness_value(routes, dist_matrix) # 计算所有路径的适应度
    best_route_index = fitness_values.argmax() # 得到适应度最高的路径所在的位置
    if fitness_values[best_route_index] > best_fitness: # 若这一代的最佳适应度超过上一代，即这一代的路径长度低于上一代
        not_improve_time = 0 # 不进行进化
        best_route, best_fitness = routes[best_route_index], fitness_values[best_route_index]  # 保存最优路线及其适应度
    else:
        not_improve_time += 1
    if (i + 1) % 200 == 0:
        print('epoch: {}, 当前最优路线距离： {}'.format(i + 1, 1 / get_route_fitness_value(best_route, dist_matrix)))
    if not_improve_time >= 2000:
        print('连续2000次迭代都没有改变最优路线，结束迭代')
        break
print('最优路线为：')
print(best_route)
print('总距离为： {}'.format(1 / get_route_fitness_value(best_route, dist_matrix)))
end = time.time()
print('耗时: {}s'.format(end - start))